In [ ]:
import torch
import numpy as np
import pandas as pd
import copy
import torch.nn.functional as F
import scipy.sparse as sp
from torch_geometric.nn import GCNConv,GATConv,SAGEConv
from torch_geometric.datasets import Planetoid
from torch.nn import Linear
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import datetime
from torch_geometric.data import Data

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

In [ ]:
torch.cuda.set_device(1)
torch.cuda.current_device()

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 处理边关系
import json
edges_file = "/public/home/liujunwu/workdir/scripts/GNN_Reactome/reaction_file/reactome_reaction.edges.txt"
edges = pd.read_csv(edges_file,header=None,sep="\t")
edges.columns = ["edge1","edge2","edge1_type","edge2_type"] 
uniq_output_nodes = sorted(list(set(edges["edge2"].tolist()))) ## 取得唯一入度的点
uniq_nodes = sorted(list(set(edges["edge1"].tolist()) | set(edges["edge2"].tolist())))
nodes_index_dict = dict(map(reversed, enumerate(uniq_nodes))) ## 将ID和对应的index存为字典
#with open("/public/home/liujunwu/workdir/scripts/GNN_Reactome/reaction_file/reactome_reaction.uniqnodes.json", "w", encoding='utf-8') as f:
    # json.dump(dict_, f)  # 写为一行
    #json.dump(nodes_index_dict, f, indent=2, sort_keys=True, ensure_ascii=False)  # 写为多行
# index_nodes_dict = dict(enumerate(uniq_nodes))  ## 将index和对应的ID存为字典
print (len(uniq_nodes))
print (nodes_index_dict["R-HSA-5340130"])
edges["edge1"] = edges["edge1"].map(nodes_index_dict)
edges["edge2"] = edges["edge2"].map(nodes_index_dict)

In [ ]:
print (len(uniq_output_nodes))
print (edges.iloc[0:4,0:4])
print (edges.loc[0,"edge1"])
sub_node_edges = []
for index, node in enumerate(uniq_output_nodes):  
    sub_node = edges[edges["edge2"] == nodes_index_dict[node]]
    for e in range(sub_node.shape[0]):
        sub_node_edges.append([sub_node.iat[e,0],sub_node.iat[e,1]])
edges_torch = torch.tensor(sub_node_edges,dtype = torch.int64).T

In [ ]:
edges_torch[1,1]


In [ ]:
print (edges_torch.shape)
torch.save(edges_torch,"/public/home/liujunwu/workdir/scripts/GNN_Reactome/reaction_file/Reactome_reaction_edges.pt")

In [ ]:
## 读取reaction包含的基因
ReactionGeneFile = "/public/home/liujunwu/workdir/scripts/GNN_Reactome/reaction_file/reactome_reaction.ProteinReactionNodes.txt"
ReactionGene = pd.read_csv(ReactionGeneFile,header=None,sep='\t')
ReactionGene.columns = ["Reaction","Gene"]
ReactionGene_dict = dict(zip(ReactionGene["Reaction"],ReactionGene["Gene"])) ## Reaction 对应的Gene
print (ReactionGene_dict['R-HSA-1112666'])
del ReactionGene

In [ ]:
GTEx_expfile = "/public/home/liujunwu/workdir/scripts/GNN_Reactome/GTEx_exp/GTEx_exp.csv" ## 读取GTEx表达量文件
#GTEx_expfile = "/public/home/liujunwu/workdir/scripts/GNN_Reactome/GTEx_exp/GTEx_exp_down5000.csv" ## 小数据集测试

GTEx_exp = pd.read_csv(GTEx_expfile,header=0,sep=",")
GTEx_genelist = GTEx_exp.columns

In [ ]:
GTEx_exp.shape

In [ ]:
## GTEx的样本标签
label_file = "/public/home/liujunwu/workdir/scripts/GNN_Reactome/GTEx_exp/GSEA/GTEx.sample_label.txt"
sample_labels = pd.read_csv(label_file,header=0,sep="\t")
sample_full_labels = sorted(list(set(sample_labels["SMTS"].tolist()))) ## 唯一值
sample_full_labels_dict = dict(zip(sample_full_labels,range(len(sample_full_labels)))) ## label对应的index
list_labels = []
print (sample_full_labels_dict)
sample_dict = dict(zip(sample_labels["SAMPID"],sample_labels["SMTS"]))
print (sample_full_labels_dict[sample_dict["GTEX-1GF9W-1326-SM-7P8PX"]])
#for x in range(len(sample_full_labels)):
#    list_labels.append([x])
#list_labels_tensor = torch.tensor(list_labels,dtype=torch.int64)
#print (list_labels_tensor[sample_full_labels_dict[sample_dict["GTEX-1GF9W-1326-SM-7P8PX"]]])

In [ ]:
## 测试
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
pca = PCA(n_components=2)
print (GTEx_exp.loc[0:5,["Sample","Tissue","WASH7P"]])
x = GTEx_exp.iloc[:,2:10].values
#x = StandardScaler().fit_transform(x)
principalComponents = pca.fit_transform(x)
print (type(principalComponents))
print (principalComponents)
#principalDf = pd.DataFrame(data = principalComponents,columns = ["PCA1","PCA2"])
#print (principalDf.iloc[0:3,0:3])


In [ ]:
principalComponents

In [ ]:
#a = pd.DataFrame(np.zeros((4,1)),columns=["PCA1"])
sample_reaction_pca = pd.DataFrame(size=(GTEx_exp.shape[0],0))
print (sample_reaction_pca)
#print (a)

In [ ]:
## 对于每一个reaction , PCA 得到的主成分 PCA1值
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
pca = PCA(n_components=2)
print(datetime.datetime.now().strftime('%Y-%m-%d  %H:%M:%S'))
sample_reaction_pca = pd.DataFrame()
n = 0
reaction_PCA_values = []
for node in iter(uniq_nodes): ## 计算所有reaction节点 对应的PCA1 values
    #print (node)
    #node = "R-HSA-1006169"
    if (node in ReactionGene_dict.keys()):
        genelist = ReactionGene_dict[node].split(",")
        common_gene = list(set(genelist) & set(GTEx_genelist))
        ## 去除bulk中不存在的基因
        sub_exp = GTEx_exp.loc[:,common_gene]
        #print (sub_exp)
        if (sub_exp.shape[1] >= 2):
            #sub_exp = sub_exp.values
            #sub_exp = StandardScaler().fit_transform(sub_exp.values)
            principalComponents = pca.fit_transform(sub_exp.values)
            PCA_values =  pd.DataFrame(data = principalComponents,columns = ["PCA1","PCA2"])
            node_PCA_values = PCA_values["PCA1"]
        elif (sub_exp.shape[1] == 1): ## 如果reaction中只包含一个基因，无法做PCA，则对该基因表达量进行标准化
            sub_exp = StandardScaler().fit_transform(sub_exp.values)
            node_PCA_values = pd.DataFrame(data = sub_exp, columns=["PCA1"])
            #print (node_PCA_values)
        else:
            node_PCA_values = pd.DataFrame(np.zeros((GTEx_exp.shape[0],1)),columns=["PCA1"])
    else:
        node_PCA_values = pd.DataFrame(np.zeros((GTEx_exp.shape[0],1)),columns=["PCA1"])
    reaction_PCA_values.append(node_PCA_values)
    #sample_reaction_pca = pd.concat([sample_reaction_pca,node_PCA_values],axis=1,join='outer',ignore_index=True)
    #n += 1
    #if n >3:break
    #break
sample_reaction_pca = pd.concat(reaction_PCA_values,axis=1,join='outer',ignore_index=True)
print(datetime.datetime.now().strftime('%Y-%m-%d  %H:%M:%S'))
print (sample_reaction_pca.iloc[0:10,0:10])
print (sample_reaction_pca.shape)


In [ ]:
a = sample_reaction_pca.loc[11714,].values
b = a.reshape(1,a.shape[0])
print (b.shape)

In [ ]:
n = 0
GTEx_dataset = []
print(datetime.datetime.now().strftime('%Y-%m-%d  %H:%M:%S'))
for sample in GTEx_exp["Sample"]:
    #print (sample,n)
    sampleValue = sample_reaction_pca.loc[n,].values
    sampleValue = sampleValue.reshape(sampleValue.shape[0],1)
    sample_node_feature = Data(x=torch.tensor(sampleValue ,dtype = torch.float32),y=torch.tensor(sample_full_labels_dict[sample_dict[sample]]),edge_index = edges_torch)
    GTEx_dataset.append(sample_node_feature)
    n = n+1
print(datetime.datetime.now().strftime('%Y-%m-%d  %H:%M:%S'))
print (len(GTEx_dataset))

In [ ]:
print (GTEx_dataset[0].x)

In [ ]:
print (list_labels_tensor[sample_full_labels_dict[sample_dict["GTEX-1GF9W-1326-SM-7P8PX"]]])

In [ ]:
print (GTEx_dataset[0].x)

In [ ]:
from torch.nn import Linear
from torch_geometric.nn import GraphConv

class GNN(torch.nn.Module):
    def __init__(self, num_node_features,hidden_channels,num_classes):
        super(GNN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GraphConv(num_node_features, hidden_channels)
        self.conv2 = GraphConv(hidden_channels, hidden_channels)
        self.conv3 = GraphConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels,num_classes)
        
    def forward(self, x, edge_index, batch,edge_weight=None):
        # 1. 获得节点嵌入
        x = self.conv1(x, edge_index,edge_weight)
        x = x.relu()
        x = self.conv2(x, edge_index,edge_weight)
        x = x.relu()
        x = self.conv3(x, edge_index,edge_weight)
        
        # 2. Readout layer
        x = global_mean_pool(x, batch)   # [batch_size, hidden_channels]
        
        # 3. 分类器
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        return x

In [ ]:
?GraphConv

In [ ]:
model = GNN(num_node_features=1,hidden_channels=64,num_classes=31).to(device)
#model = NeuralNetwork().to(device)
print (model)

In [ ]:
from torch_geometric.nn import global_mean_pool
from torch_geometric.loader import DataLoader
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()
torch.set_num_threads(48)
#print (train_dataset[0])
train_dataset1 = GTEx_dataset[0:15000]
print (train_dataset1[20])
train_loader = DataLoader(train_dataset1, batch_size=32,shuffle=True)
def train():
    model.train()
    for data in train_loader:
        #print (data.y)
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index,data.batch)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
def test(loader):
    model.eval()
    correct = 0
    for data in loader:                            # 批遍历测试集数据集。
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch) # 一次前向传播
        pred = out.argmax(dim=1)   # 使用概率最高的类别
        #print (pred,data.y)
        correct += int((pred == data.y).sum())           # 检查真实标签
    return correct / len(loader.dataset)

model.train()
print(datetime.datetime.now().strftime('%Y-%m-%d  %H:%M:%S'))

for epoch in range(1, 301):
    train()
    print(datetime.datetime.now().strftime('%Y-%m-%d  %H:%M:%S'))
    train_acc = test(train_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}')

In [ ]:
print (train_dataset1[0])

In [ ]:
for name, param in model.named_parameters(): print(name, param)

In [ ]:
# 测试集
model.eval()
test_data = GTEx_dataset[15000:]
print (len(test_data))
test_loader = DataLoader(test_data, batch_size=100,shuffle=True)
test_acc = test(test_loader)
#print (test_acc.)
print(f'Test Acc: {test_acc:.4f}')

In [ ]:
# 保存
saved_dict = {
 'model': model.state_dict(),
 'opt': optimizer.state_dict()
}
torch.save(saved_dict, '/public/home/liujunwu/workdir/scripts/GNN_Reactome/reaction_file/reaction.PCA.pth.tar')

In [ ]:
torch.save(model, '/public/home/liujunwu/workdir/scripts/GNN_Reactome/reaction_file/reaction.PCA_convEdgeWeight.1018.pt')